In [8]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('./Clients_14.csv', encoding='latin1')
df2 = pd.read_csv('./Clients_5.csv', encoding='latin1')

# Concatenate df and df2
concatenated_df = pd.concat([df, df2], ignore_index=True)

# Define a function to clean the 'sexe' column
def clean_sexe(value):
    if value.lower() in ['homme', 'masculin']:
        return 'M'
    elif value.lower() in ['femme', 'féminin']:
        return 'F'
    else:
        return np.random.choice(['M', 'F'], p=[0.5, 0.5])

# Apply the cleaning function to the 'sexe' column
concatenated_df['sexe'] = concatenated_df['sexe'].apply(clean_sexe)

# Define a function to clean the 'age' column
def clean_age(value):
    if str(value) in ['-1', ' ', '?']:
        return np.random.randint(18, 82)  # Generate a random value between 18 and 81
    else:
        return value

# Apply the cleaning function to the 'age' column
concatenated_df['age'] = concatenated_df['age'].apply(clean_age)

# Replace 'Célibataire' with 'Celibataire' in the 'situationFamiliale' column
concatenated_df['situationFamiliale'] = concatenated_df['situationFamiliale'].replace('Célibataire', 'Celibataire')

# Convert 'taux' column to numeric
concatenated_df['taux'] = pd.to_numeric(concatenated_df['taux'], errors='coerce')

# Replace specified values with NaN
concatenated_df['taux'].replace([-1, -1.0, ' ', '?', np.nan], np.nan, inplace=True)

# Calculate the mean value
mean_taux = round(concatenated_df['taux'].mean(), 0)

# Fill NaN values with the mean
concatenated_df['taux'].fillna(mean_taux, inplace=True)

# Replace specified values in 'nbEnfantsAcharge' with 0
concatenated_df['nbEnfantsAcharge'].replace(['-1', ' ', '?'], 0, inplace=True)

# Convert '2eme voiture' column values to uppercase
concatenated_df['2eme voiture'] = concatenated_df['2eme voiture'].str.upper()

# Check if values are not 'TRUE' or 'FALSE', and replace them with TRUE or FALSE at random
for index, value in concatenated_df['2eme voiture'].items():
    if value not in ['TRUE', 'FALSE']:
        concatenated_df.at[index, '2eme voiture'] = np.random.choice(['TRUE', 'FALSE'])

# Define a function to clean the 'situationFamiliale' column
def clean_situation(value):
    if value.lower() in ['seule', 'seul', 'célibataire']:
        return 'Celibataire'
    elif value.lower() == 'divorcée':
        return 'Divorcee'
    elif value.lower() == 'marié(e)':
        return 'Marie(e)'
    elif value.lower() in [' ', '?', 'n/d']:
        return np.random.choice(['Celibataire', 'En Couple', 'Marie(e)', 'Divorcee'])
    else:
        return value

# Apply the cleaning function to the 'situationFamiliale' column
concatenated_df['situationFamiliale'] = concatenated_df['situationFamiliale'].apply(clean_situation)

concatenated_df.rename(columns={'2eme voiture': 'deuxiemeVoiture'}, inplace=True)

# Display the cleaned data
# print("\nCleaned Data:")
# print(concatenated_df)

# Filter rows based on immatriculation format
pattern = r'^\d{4}\s[A-Z]{2}\s\d{2}$'
concatenated_df = concatenated_df[concatenated_df['immatriculation'].str.match(pattern, na=False)]

# Delete one of the duplicate rows while keeping the first occurrence
concatenated_df = concatenated_df.drop_duplicates(subset=['immatriculation'], keep='first')

# Set the 'ID' column using a different method
concatenated_df.insert(0, 'ID', range(1, len(concatenated_df) + 1))

# Save the cleaned data to a new CSV file if needed
concatenated_df.to_csv('Cleaned_Clients.csv', index=False)
